Steps of ABC:

1- Generate initial randomized population an evaluate them.

2- Movement of recruited bees.

3- Send onlooker bees. (Calculate probabilities -> Select site -> Generate new solution -> Evaluation)

4- If there is a site which is reached to the the number of unproceeded counts, that site will be replaced randomly with another one.

5- If end conditions are not met yet, return to the step 2, else the end.

This algorithm just works for continuous space.


In [1]:
#import libraries
import numpy as np

In [2]:
#define function

def sphere(x):
    return np.sum(np.power(x,2))

In [ ]:
#problem definition
def costFunction(x):
    return sphere(x)

nVar = 5    #number of decision variable
varSize = nVar #decision variable matrix size
varMin = -10    # decision variable lower bound,  can be changed to array
varMax = 10     #decision variable upper bound, can be changed to array

